In [ ]:
!pip install anthropic pandas

In [ ]:
from google.colab import userdata
import os

# Secretsからキーを取得
api_key = userdata.get('ANTHROPIC_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = api_key  # ライブラリが環境変数を参照する場合

In [ ]:
import pandas as pd
import re
from anthropic import Anthropic
import time # time.sleepを使うために必要
from io import StringIO
import json
from google.colab import files # Colabでファイルアップロード機能を使うためのライブラリ
import asyncio
from anthropic import AsyncAnthropic # 非同期処理用のクライアント
from tqdm.asyncio import tqdm # 非同期処理に対応した進捗バー
from tqdm import tqdm # 進捗表示

# --- 設定 ---
API_KEY = os.environ.get("ANTHROPIC_API_KEY")
MODEL_NAME = "claude-3-haiku-20240307"
CONCURRENCY_LIMIT = 50 # 同時リクエスト数の制限
MAX_RETRIES = 3 # 最大リトライ回数

# APIキーのチェック
if not API_KEY:
    print("🚨 警告: ANTHROPIC_API_KEYが環境変数に設定されていません。分類処理は実行されません。")

In [ ]:
# --- データプレパレーションとデータ読み込みと初期設定 ---

try:
    uploaded_file_name = 'Retail.OrderHistory.3_add_OrderID_2021.csv'

    # ファイルを読み込み（エンコーディングエラー回避のため試行）
    try:
        df = pd.read_csv(uploaded_file_name)
    except UnicodeDecodeError:
        df = pd.read_csv(uploaded_file_name, encoding='shift_jis', on_bad_lines='skip')

    print(f"✅ ファイル {uploaded_file_name} を読み込みました。")

except FileNotFoundError:
    print("❌ エラー: ファイルがアップロードされているか、ファイル名が正しいか確認してください。")
    raise # 処理を中断

# 必要な列の定義
REQUIRED_COLUMNS = ['Order ID','Order Date', 'Total Owed', 'ASIN', 'Product Name']
if not all(col in df.columns for col in REQUIRED_COLUMNS):
    print(f"❌ エラー: CSVファイルに必要な列 ({REQUIRED_COLUMNS}) が見つかりません。")
    print(f"存在する列: {list(df.columns)}")
    raise # 処理を中断

# 必要な列のみに絞り、名前を変更
df_processed = df.rename(columns={'Total Owed': '価格', 'Product Name': '書籍名'}).copy()
df_processed = df_processed[['Order ID','Order Date', 'ASIN', '書籍名', '価格']]

# NaN値や空行を念のため除去
df_processed.dropna(subset=['書籍名'], inplace=True)


# --- 価格クリーンアップ関数 ---

def clean_price(price_str):
    """価格文字列から数値のみを抽出し、int型に変換する関数"""
    if isinstance(price_str, str):
        # 数字以外（カンマ、円マークなど）をすべて削除
        cleaned_str = re.sub(r'[^\d]', '', price_str.replace('"', ''))
        try:
            if cleaned_str:
                # floatを経由してintに変換（整数化）
                return int(float(cleaned_str))
            return None
        except ValueError:
            return None
    return None

# 新しい「価格 (数値)」列を作成
df_processed['価格 (数値)'] = df_processed['価格'].apply(clean_price)
df_processed.drop(columns=['価格'], inplace=True) # 元の文字列列は削除

print("\n--- データ準備完了（最初の5件） ---")
print(df_processed.head())
print("-" * 30)

In [ ]:
# --- 直列処理：LLM推論による分類と勘定科目の同時実行 ---

# 勘定科目リストと分類の推論ポイントを統合した指示
CLASSIFICATION_AND_ACCOUNT_INSTRUCTIONS = """
あなたは、ユーザーの購入データを「分類 (事業用/私用)」と「勘定科目」の2つのフィールドに推論・分類する会計アシスタントです。
以下の指示とリストに基づき、必ず**JSON形式**で回答してください。他の説明や付加情報は一切不要です。

【分類 (事業用/私用) の推論ポイント】
・技術書、専門書、思考・教養書（例：哲学、論理学、歴史、**仏教、宗教**など）、PC周辺機器、作業環境維持用品は「事業用」とする。
・上記に該当しないもの、趣味や一般教養とみなされるものは「私用」とする。

【勘定科目リスト】
・新聞図書費: 技術書、専門書、ビジネス教養書など、情報収集のための費用。
・消耗品費: ケーブル、電源タップ、消毒液、その他PC周辺小物など、使用により消耗する物品。
・研究開発費: 専門的な開発ツール、高度な技術書、試作品の材料費。
・雑費: 上記に該当しない少額な費用。
・（分類が私用の場合は、勘定科目は「私用」とすること）

---
【出力形式の厳守】
{
  "分類": "[事業用 または 私用]",
  "勘定科目": "[新聞図書費, 消耗品費, 研究開発費, 雑費, 私用 のいずれか]"
}
"""

def infer_two_fields(title):
    """Claude 3 Haikuに分類と勘定科目の両方をリクエストし、JSONで受け取る関数"""

    # 🚨 API_KEYとDFが存在しない場合のエラーチェック (既存コードから継承)
    if not API_KEY or title is None:
        return {"分類": "APIキー未設定", "勘定科目": "APIキー未設定"} if not API_KEY else {"分類": "データ欠損", "勘定科目": "データ欠損"}

    client = Anthropic(api_key=API_KEY)

    system_prompt = CLASSIFICATION_AND_ACCOUNT_INSTRUCTIONS
    user_prompt = f"以下の書籍/物品のタイトルを分類し、勘定科目を推論してください: 「{title}」"

    try:
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=100, # JSON出力のためにトークン数を増やす
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_prompt}
            ]
        )

        # JSON文字列を抽出してパース
        json_text = response.content[0].text.strip()
        result = json.loads(json_text)

        # 結果の検証
        if result['分類'] in ["事業用", "私用"] and result['勘定科目'] in ["新聞図書費", "消耗品費", "研究開発費", "雑費", "私用"]:
            return result
        else:
            print(f"警告: 「{title}」の結果が不正です: {result}")
            return {"分類": "分類エラー", "勘定科目": "分類エラー"}

    except Exception as e:
        # print(f"APIエラー for {title}: {e}") # エラーログ出力回避
        return {"分類": "APIエラー", "勘定科目": "APIエラー"}

# -----------------------------------------------------
# データフレームへの適用と結果の処理
# -----------------------------------------------------

df_target = df_processed.copy() # 元のデータフレーム df をコピー

# 実際の分類処理の実行（APIキーが設定されている場合のみ）
if API_KEY:
    # 行ごとに推論関数を適用し、結果（JSONオブジェクト）をSeriesとして受け取る
    inferred_results = df_target['書籍名'].apply(infer_two_fields)

    # JSON結果を展開し、DataFrameの新しい列として追加
    df_target['分類'] = inferred_results.apply(lambda x: x.get('分類', '不明'))
    df_target['勘定科目'] = inferred_results.apply(lambda x: x.get('勘定科目', '不明'))
else:
    df_target['分類'] = "APIキー未設定"
    df_target['勘定科目'] = "APIキー未設定"


print("\n--- 最終結果（最初の30件） ---")
# 不要になった元の「価格」列を削除（もし存在すれば）し、結果を表示
if '価格' in df_target.columns:
    df_target = df_target.drop(columns=['価格'])

print(df_target.head(30))

# 必要であれば、結果を新しいCSVとしてダウンロード
df_target.to_csv('classified_order_history_with_account.csv', index=False, encoding='utf-8-sig')
files.download('classified_order_history_with_account.csv')

In [ ]:
# --- 直列処理 ---

# --- ステップ 3: LLM推論による分類実行 ---

# あなたの推論ポイントをプロンプトに組み込む
CLASSIFICATION_INSTRUCTIONS = """
あなたは、ユーザーの購入した書籍や物品を「事業用」か「私用」かに分類する会計アシスタントです。
以下の推論ポイントを厳守し、**出力は「事業用」または「私用」のどちらか一言のみ**にしてください。他の説明や付加情報は一切不要です。

【推論のポイント】
・技術系（事業用）: LangChain, PyTorch, TensorFlow, x86, デバッガ, TCP/IP, ネットワーク分析, 量子, 統計力学, 数学, 物理学など、直接的な技術スキルや学習に関連するものは「事業用」とする。
・哲学・思考系（事業用）: ニュー・アトランティス, ユートピア, ロック, ヒューム, 断片と全体など、事業の根幹となる深い思考や教養に資するものは「事業用」とする。仏教、宗教も事業用とする。
・物理的備品・インフラ（事業用）: Anker PowerPort III 65W Podやエレコム(ELECOM) 電源タップなど、PC周辺機器や作業環境の維持に必要なものは「事業用」とする。
・上記に該当しないもの、趣味や一般教養とみなされるものは「私用」とする。
"""

def classify_item(title):
    """Claude 3 Haikuに分類をリクエストする関数"""
    if not API_KEY or title is None:
        return "APIキー未設定" if not API_KEY else "データ欠損"

    client = Anthropic(api_key=API_KEY)

    system_prompt = CLASSIFICATION_INSTRUCTIONS
    user_prompt = f"以下の書籍/物品のタイトルを分類してください: 「{title}」"

    try:
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=5,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_prompt}
            ]
        )
        classification = response.content[0].text.strip()

        # 結果が意図通りかチェックし、不正な出力の場合はエラーとして処理
        if classification in ["事業用", "私用"]:
             return classification
        else:
            print(f"警告: 「{title}」の分類結果が不正です: {classification}")
            return "分類エラー"

    except Exception as e:
        # print(f"APIエラー for {title}: {e}") # 処理が多い場合、エラーログが大量に出るのを避ける
        return "APIエラー"

# データフレームの行ごとに分類関数を適用
# 実行時間短縮のため、ここでは最初の30件に絞る例としています。全件処理する場合は [0:30] を削除してください。
df_target = df.copy()

# 実際の分類処理の実行（APIキーが設定されている場合のみ）
if API_KEY:
    df_target['分類'] = df_target['書籍名'].apply(classify_item)
else:
    df_target['分類'] = "APIキー未設定"


print("\n--- 最終結果（最初の30件） ---")
# 不要になった元の「価格」列を削除し、結果を表示
df_final = df_target.drop(columns=['価格'])
print(df_final)

# 必要であれば、結果を新しいCSVとしてダウンロード
df_final.to_csv('classified_order_history.csv', index=False, encoding='utf-8-sig')
files.download('classified_order_history.csv')

In [ ]:
# --- 並列スレッド処理 ---

async def classify_item_async(title, client, index, semaphore):
    """セマフォを使って並行数を制限しながらAPIにリクエストする関数"""
    if not API_KEY or title is None:
        return index, "APIキー未設定" if not API_KEY else "データ欠損"

    user_prompt = f"以下の書籍/物品のタイトルを分類してください: 「{title}」"

    # セマフォが空くまで待機し、リソースを確保
    async with semaphore:
        try:
            response = await client.messages.create(
                model=MODEL_NAME,
                max_tokens=5,
                system=CLASSIFICATION_INSTRUCTIONS,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            classification = response.content[0].text.strip()

            if classification in ["事業用", "私用"]:
                return index, classification
            else:
                return index, "分類エラー"

        except Exception as e:
            # ネットワークエラーなどが発生した場合
            # print(f"APIリクエストエラー発生: {e}") # ログが大量に出るのを防ぐためコメントアウト
            return index, "APIエラー"


async def process_all_data(df):
    """全データを並列処理するためのメイン非同期関数"""
    if not API_KEY:
        return [(index, "APIキー未設定") for index in df.index]

    client = AsyncAnthropic(api_key=API_KEY)
    semaphore = asyncio.Semaphore(CONCURRENCY_LIMIT) # 並行数制限

    tasks = [
        classify_item_async(title, client, index, semaphore)
        for index, title in df['書籍名'].items()
    ]

    print(f"\n💡 全データ {len(df)} 件の分類を、並行数 {CONCURRENCY_LIMIT} で開始します。")

    results = await tqdm.gather(*tasks, desc="並行処理の進行")

    return results


# --- 実行と結果の統合 ---

# 非同期関数を実行し、結果を取得 (Colab/Jupyter環境で実行)
results = await process_all_data(df_processed)

# 結果の統合
classification_map = {index: classification for index, classification in results}
df_processed['分類'] = df_processed.index.map(classification_map)


# --- 最終出力とダウンロード ---

print("\n--- 最終結果（最初の5件） ---")
print(df_processed.head())

output_filename = 'classified_order_history_controlled_async.csv'
df_processed.to_csv(output_filename, index=False, encoding='utf-8-sig')
files.download(output_filename)

print(f"\n✅ 処理が完了しました。ファイル '{output_filename}' のダウンロードが開始されます。")
print(f"\n★ エラー率の確認: APIエラー件数: {df_processed['分類'].value_counts().get('APIエラー', 0)} 件 / {len(df_processed)} 件中")

In [ ]:
# --- 指数バックオフ処理 ---

# --- リトライ機能付き分類関数の定義 ---

def classify_item_with_retry(title):
    """リトライロジックを組み込んだ分類関数（直列処理）"""
    if not API_KEY or title is None:
        return "APIキー未設定" if not API_KEY else "データ欠損"

    # 同期クライアントを使用
    client = Anthropic(api_key=API_KEY)
    user_prompt = f"以下の書籍/物品のタイトルを分類してください: 「{title}」"

    for i in range(MAX_RETRIES):
        try:
            # --- APIコール ---
            response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=5,
                system=CLASSIFICATION_INSTRUCTIONS,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            classification = response.content[0].text.strip()

            # 成功したらすぐに結果を返す
            if classification in ["事業用", "私用"]:
                 return classification
            else:
                 # モデルが出力形式を間違えた場合もリトライはしない
                 return "分類エラー"

        except Exception as e:
            # 最終リトライだった場合はエラーを確定させる
            if i == MAX_RETRIES - 1:
                # print(f"最終リトライ失敗: {title} - {e}")
                return "APIエラー"

            # 最終リトライではない場合、指数バックオフで待機
            delay = 2 ** i # 1, 2, 4秒と待機時間が指数関数的に増加
            print(f"⚠️ APIエラー発生（リトライ {i+1}回目）。{delay}秒待機します: {title[:30]}...")
            time.sleep(delay)

    return "APIエラー" # 念のための安全策

# --- 実行と結果の統合 ---

# df_processed が存在することを確認
if 'df_processed' not in locals():
    print("エラー: df_processed DataFrameが見つかりません。ステップ2のデータ準備（前回の回答）を実行してください。")
    raise

print(f"\n💡 全データ {len(df_processed)} 件の分類を、リトライ機能付きの直列処理で開始します。（処理時間は約11分程度かかる見込みです）")

# df.apply() で直列処理を実行し、tqdmで進捗を表示
tqdm.pandas(desc="直列処理の進行")
df_processed['分類'] = df_processed['書籍名'].progress_apply(classify_item_with_retry)


# --- 最終出力とダウンロード ---

print("\n--- 最終結果（最初の5件） ---")
print(df_processed.head())

output_filename = 'classified_order_history_retry_serial.csv'
df_processed.to_csv(output_filename, index=False, encoding='utf-8-sig')
files.download(output_filename)

print(f"\n✅ 処理が完了しました。ファイル '{output_filename}' のダウンロードが開始されます。")
error_count = df_processed['分類'].value_counts().get('APIエラー', 0)
print(f"\n★ 最終エラー率の確認: APIエラー件数: {error_count} 件 / {len(df_processed)} 件中")